<a href="https://colab.research.google.com/github/chunhan931013-bit/Tch-/blob/main/POCUS_(Point_of_Care_Ultrasound)_and_hemodynamics_calculator_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

def get_float(prompt):
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("Please enter a valid number.")

def get_str(prompt):
    return input(prompt).strip()

print("--- GEMINI POCUS CALCULATOR ---")
print("Please key in the measurements one by one.\n")

# 1. DATA ENTRY
weight = get_float("Body weight (kg): ")
height = get_float("Height (cm): ")
age = get_float("Age: ")
gender = get_str("Gender (M/F): ")
hr = get_float("Heart Rate (bpm): ")

print("\n--- CARDIAC ---")
lvot_d = get_float("LVOT diameter (cm): ")
lvot_vti = get_float("LVOT VTI (cm): ")
lvesd = get_float("LVESD (cm): ")
lvedd = get_float("LVEDD (cm): ")
epss = get_float("EPSS (mm): ")
mapse = get_float("MAPSE (mm): ")
tapse = get_float("TAPSE (mm): ")
rv_basal = get_float("RV basal diameter (cm): ")
lv_basal = get_float("LV basal diameter (cm): ")
valve = get_str("Valve abnormalities: ")
pe = get_str("Pericardial effusion: ")
ivc_max = get_float("IVC size max (cm): ")
ivc_min = get_float("IVC size min (cm): ")

print("\n--- LUNG ZONES (A/B/Shred/Hepatization) ---")
r1 = get_str("R1: ")
r2 = get_str("R2: ")
r3 = get_str("R3: ")
r4 = get_str("R4: ")
l1 = get_str("L1: ")
l2 = get_str("L2: ")
l3 = get_str("L3: ")
l4 = get_str("L4: ")

print("\n--- PLEURAL EFFUSION ---")
effusion_side = get_str("Presence of pleural effusion (Left / Right / Bilateral / None): ").lower()

# Left Effusion
l_ribs = l_pool = 0
l_nod = l_sept = l_thick = "no"
if "left" in effusion_side or "bilateral" in effusion_side:
    print(">> Left Effusion Details:")
    l_ribs = get_float("How many rib spaces: ")
    l_pool = get_float("Deepest pool (cm): ")
    l_nod = get_str("Nodularity (yes/no): ").lower()
    l_sept = get_str("Septation (yes/no): ").lower()
    l_thick = get_str("Pleural thickness >3mm (yes/no): ").lower()

# Right Effusion
r_ribs = r_pool = 0
r_nod = r_sept = r_thick = "no"
if "right" in effusion_side or "bilateral" in effusion_side:
    print(">> Right Effusion Details:")
    r_ribs = get_float("How many rib spaces: ")
    r_pool = get_float("Deepest pool (cm): ")
    r_nod = get_str("Nodularity (yes/no): ").lower()
    r_sept = get_str("Septation (yes/no): ").lower()
    r_thick = get_str("Pleural thickness >3mm (yes/no): ").lower()

others = get_str("Others important point: ")

# 2. CALCULATIONS
# BSA (Mosteller)
bsa = math.sqrt((height * weight) / 3600)

# LVOT Area (cm2) = pi * r^2
lvot_area = math.pi * ((lvot_d / 2) ** 2)

# Stroke Volume (mL) = Area * VTI
sv = lvot_area * lvot_vti

# Cardiac Output (L/min)
co = (sv * hr) / 1000

# Cardiac Index (L/min/m2)
ci = co / bsa

# IVC Indices
ivc_collapsibility = ((ivc_max - ivc_min) / ivc_max) * 100 if ivc_max > 0 else 0
ivc_distensibility = ((ivc_max - ivc_min) / ivc_min) * 100 if ivc_min > 0 else 0

# RV:LV Ratio
rv_lv_ratio = rv_basal / lv_basal if lv_basal > 0 else 0

# LVEF (Teichholz)
# V = (7.0 / (2.4 + D)) * D^3
lvedv = (7.0 / (2.4 + lvedd)) * (lvedd ** 3)
lvesv = (7.0 / (2.4 + lvesd)) * (lvesd ** 3)
lvef = ((lvedv - lvesv) / lvedv) * 100 if lvedv > 0 else 0

# DUETS Score (Simplified approximation based on user inputs)
# D=Diaphragmatic Nodularity, U=Unilateral, E=Echogenicity, T=Thickening, S=Septation
# Note: Score 1 point for each. >=2 indicates Exudate.
def calc_duets(nod, sept, thick, side_str):
    score = 0
    if nod == "yes": score += 1 # D
    if "bilateral" not in side_str and "none" not in side_str: score += 1 # U (Unilateral)
    # E (Echogenicity) not explicitly asked, but Septation often implies complex echogenicity
    if thick == "yes": score += 1 # T
    if sept == "yes": score += 1 # S
    return score

duets_l = calc_duets(l_nod, l_sept, l_thick, effusion_side) if "left" in effusion_side or "bilateral" in effusion_side else 0
duets_r = calc_duets(r_nod, r_sept, r_thick, effusion_side) if "right" in effusion_side or "bilateral" in effusion_side else 0

# 3. GENERATE SUMMARY
print("\n" + "="*30)
print("       CLINICAL SUMMARY")
print("="*30)
print(f"Patient: {age:.0f}y {gender} | BSA: {bsa:.2f} m2")
print(f"Hemodynamics:")
print(f"  - Stroke Volume: {sv:.1f} mL")
print(f"  - Cardiac Output: {co:.2f} L/min")
print(f"  - Cardiac Index: {ci:.2f} L/min/m2")
print(f"  - LVOT Area: {lvot_area:.2f} cm2")
print(f"  - LVEF (Teichholz): {lvef:.1f}%")
print(f"  - RV:LV Ratio: {rv_lv_ratio:.2f}")

print(f"Volume Status:")
print(f"  - IVC Collapsibility (Spon): {ivc_collapsibility:.1f}%")
print(f"  - IVC Distensibility (Mech): {ivc_distensibility:.1f}%")

print(f"Lung Ultrasound:")
print(f"  - Right: {r1}, {r2}, {r3}, {r4}")
print(f"  - Left:  {l1}, {l2}, {l3}, {l4}")

if "left" in effusion_side or "bilateral" in effusion_side:
    print(f"Left Effusion:")
    print(f"  - Size: {l_ribs} ribs, {l_pool} cm depth")
    print(f"  - DUETS Score (Approx): {duets_l}/5 (Score >=2 suggest Exudate)")

if "right" in effusion_side or "bilateral" in effusion_side:
    print(f"Right Effusion:")
    print(f"  - Size: {r_ribs} ribs, {r_pool} cm depth")
    print(f"  - DUETS Score (Approx): {duets_r}/5 (Score >=2 suggest Exudate)")

print("\nInterpretation Notes:")
if ci < 2.2: print("! Low Cardiac Index (< 2.2) - Consider Shock/Low Flow State")
if lvef < 50: print(f"! Reduced LVEF ({lvef:.1f}%)")
if ivc_collapsibility > 50: print("! IVC Collapsibility > 50% - Suggests Fluid Responsiveness (in spontaneous breathing)")
if ivc_distensibility > 18: print("! IVC Distensibility > 18% - Suggests Fluid Responsiveness (in mech ventilation)")
if duets_l >= 2 or duets_r >= 2: print("! High DUETS Score - High likelihood of Exudative Effusion")
print("="*30)